# 2010 census pops from csv

This version starts with a csv that is the same as an excel assignment.

Hidalgo County was fixed to remove the annotation on total populaton.

In [1]:
import agate
import warnings
from decimal import Decimal
warnings.filterwarnings('ignore')

In [2]:
column_names = ['Id', 'Id2', 'Geography',
                'Total pop', 'Hispanic', 'Not Hispanic', 'White', 'Black',
                'Indian', 'Asian', 'Hawaiian', 'Other', 'Two or more']
column_types = [agate.Text(), agate.Text(), agate.Text(), 
                agate.Number(), agate.Number(), agate.Number(), agate.Number(), agate.Number(),
                agate.Number(), agate.Number(), agate.Number(), agate.Number(), agate.Number(), ]

raw = agate.Table.from_csv(
    '../data/DEC_10_SF1_TX_County_population.csv',
    column_names,
    column_types,
    skip_lines=2)

In [3]:
print(raw)

| column       | data_type |
| ------------ | --------- |
| Id           | Text      |
| Id2          | Text      |
| Geography    | Text      |
| Total pop    | Number    |
| Hispanic     | Number    |
| Not Hispanic | Number    |
| White        | Number    |
| Black        | Number    |
| Indian       | Number    |
| Asian        | Number    |
| Hawaiian     | Number    |
| Other        | Number    |
| Two or more  | Number    |



In [15]:
simpleperc = raw.compute([
        ('test_perc', agate.Formula(agate.Number(), lambda row: row['White'] / row['Total pop']))
    ])
simpleperc.select('test_perc').print_table(3)

simpleround = simpleperc.compute([
    ('test_round', agate.Formula(agate.Number(), lambda row: row['test_perc'].quantize(Decimal('0.01'))))
])
simpleround.select('test_round').print_table(3)

| test_perc |
| --------- |
|    0.479… |
|    0.633… |
|    0.706… |
|       ... |
| test_round |
| ---------- |
|       0.48 |
|       0.63 |
|       0.71 |
|        ... |


In [4]:
# this is used by the computer below
# It does the math for the percentage
# then rounds it with the .quantize
def make_percentage(row):
    return lambda r: ((r[row] / r['Total pop']) * 100).quantize(Decimal('0.01'))


In [5]:
# this creates a new column of percentage of race
percentages = raw.compute([
    ('Hispanic %', agate.Formula(agate.Number(), make_percentage('Hispanic'))),
        
#     ('Hispanic %', agate.Formula(agate.Number(), lambda r: (r['Hispanic'] / r['Total pop']) * 100)),
    ('White %', agate.Formula(agate.Number(), make_percentage('White'))),
    ('Black %', agate.Formula(agate.Number(), make_percentage('Black'))),
    ('Indian %', agate.Formula(agate.Number(), make_percentage('Indian'))),
    ('Asian %', agate.Formula(agate.Number(), make_percentage('Asian'))),
    ('Hawaiian %', agate.Formula(agate.Number(), make_percentage('Hawaiian'))),
    ('Other %', agate.Formula(agate.Number(), make_percentage('Other'))),
    ('Two or more %', agate.Formula(agate.Number(), make_percentage('Two or more'))),
])

# I wish I could turn the above into a function and loop through
# an origina list of columns. Part of what I don't know is how to
# create a new column_name from the column passed in, among other issues

In [7]:
races = {
    'Hispanic %',
    'White %',
    'Black %',
    'Asian %',
    'Hawaiian %',
    'Other %',
    'Two or more %',
#     'Hispanic %r'
}

def print_winners(race):
    for race in races:
        print('County with highest {}\n'.format(race))
        percentages.select(['Geography', race]).order_by(race, reverse=True).print_table(3)
        print('\n')

print_winners(races)

County with highest Two or more %

| Geography            | Two or more % |
| -------------------- | ------------- |
| Coryell County, T... |          3.44 |
| Bell County, Texas   |          3.22 |
| Lamar County, Texas  |          2.26 |
| ...                  |           ... |


County with highest Hispanic %

| Geography            | Hispanic % |
| -------------------- | ---------- |
| Webb County, Texas   |      95.74 |
| Maverick County, ... |      95.68 |
| Starr County, Texas  |      95.68 |
| ...                  |        ... |


County with highest Asian %

| Geography            | Asian % |
| -------------------- | ------- |
| Fort Bend County,... |   16.87 |
| Collin County, Texas |   11.16 |
| Denton County, Texas |    6.50 |
| ...                  |     ... |


County with highest Other %

| Geography            | Other % |
| -------------------- | ------- |
| Winkler County, T... |    0.60 |
| Armstrong County,... |    0.58 |
| DeWitt County, Texas |    0.48 |
| ...     